<a href="https://colab.research.google.com/github/rishuatgithub/MLPy/blob/master/Perceptrons_Basic_Understanding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np

import tensorflow_hub as hub
import tensorflow_datasets as ds

In [3]:
## general checks

print("Tensor Flow Version : {}".format(tf.__version__))
print("Eager Mode : {}".format(tf.executing_eagerly()))
print("Hub Version : {}".format(hub.__version__))
print("GPU availablility : {}".format(True if tf.config.experimental.list_physical_devices("GPU") else False))

Tensor Flow Version : 2.2.0
Eager Mode : True
Hub Version : 0.8.0
GPU availablility : False


## IMDB Movie Review : Text Classification

#### Step 1: Download the dataset of IMDB review from TF Dataset library

In [4]:
## check if the dataset holds the imdb dataset or not
"imdb_reviews" in ds.list_builders()

True

In [5]:
## Load the dataset

if "imdb_reviews" in ds.list_builders():
  imdb_ds = ds.load("imdb_reviews")

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete69G6R7/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete69G6R7/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete69G6R7/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [6]:
for key,value in imdb_ds.items():
  print(key, value)

test <DatasetV1Adapter shapes: {label: (), text: ()}, types: {label: tf.int64, text: tf.string}>
train <DatasetV1Adapter shapes: {label: (), text: ()}, types: {label: tf.int64, text: tf.string}>
unsupervised <DatasetV1Adapter shapes: {label: (), text: ()}, types: {label: tf.int64, text: tf.string}>


In [0]:
### Using TF DS to split the training data into 60:40 validation dataset.

if "imdb_reviews" in ds.list_builders():

  train_data, validation_data, test_data = ds.load(name="imdb_reviews",split=('train[:60]','train[60:]','test'),as_supervised=True)

In [8]:
type(train_data)

tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter

In [10]:
## Explore the data set

train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [11]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

#### Step 2 : Building the Model Layers

![Model Flow](https://drive.google.com/file/d/1Max96vjkGZ1_bDAhqZzUAJghbR2WvCpV/view?usp=sharing)

In [14]:
## Embedding the word from TF Hub pre-build BoW

embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

hub_layer = hub.KerasLayer(embedding,trainable=True,input_shape=[],dtype=tf.string)

hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [17]:
### Define the layers

model = tf.keras.Sequential(name='IMDBSEQModel')
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "IMDBSEQModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense_4 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [0]:
### Define the Loss Function and optimizer

model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

#### Step 3 : Train the Model

In [20]:
history = model.fit(train_data.shuffle(10000).batch(512), epochs=20, validation_data=validation_data.batch(512), verbose=1)

Epoch 1/20
1/1 [==============================] - 3s 3s/step - loss: 2.7871 - accuracy: 0.5333 - val_loss: 2.7740 - val_accuracy: 0.4999
Epoch 2/20
1/1 [==============================] - 3s 3s/step - loss: 2.6883 - accuracy: 0.5333 - val_loss: 2.6785 - val_accuracy: 0.4999
Epoch 3/20
1/1 [==============================] - 3s 3s/step - loss: 2.5898 - accuracy: 0.5333 - val_loss: 2.5840 - val_accuracy: 0.4999
Epoch 4/20
1/1 [==============================] - 3s 3s/step - loss: 2.4916 - accuracy: 0.5333 - val_loss: 2.4907 - val_accuracy: 0.4999
Epoch 5/20
1/1 [==============================] - 3s 3s/step - loss: 2.3943 - accuracy: 0.5333 - val_loss: 2.3986 - val_accuracy: 0.4999
Epoch 6/20
1/1 [==============================] - 3s 3s/step - loss: 2.2979 - accuracy: 0.5333 - val_loss: 2.3077 - val_accuracy: 0.4999
Epoch 7/20
1/1 [==============================] - 3s 3s/step - loss: 2.2029 - accuracy: 0.5333 - val_loss: 2.2179 - val_accuracy: 0.4999
Epoch 8/20
1/1 [=========================

#### Step 4 : Evaluate the Model

In [23]:
results = model.evaluate(test_data.batch(512),verbose=2)

for name, value in zip(model.metrics_names, results):
  print("{} {}".format(name,value))

49/49 - 3s - loss: 1.1862 - accuracy: 0.5019
loss 1.186212182044983
accuracy 0.5019199848175049
